In [1]:
import math
import cv2
import numpy as np
from time import time
import mediapipe as mp
from pygame import mixer
import tempfile
from gtts import gTTS
import requests
import pygame
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

pygame 2.1.3 (SDL 2.0.22, Python 3.8.15)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [11]:
# 計算角度的副函式
def calculatexy(landmark1, landmark2):
    # 獲取所需座標
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2

    y= abs(math.floor(y1 - y2))

    return y

In [12]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    height, width, _ = image.shape
    landmarks = []
    if results.pose_landmarks:
            # 畫關鍵點在圖片上
        mp_drawing.draw_landmarks(image=image, landmark_list=results.pose_landmarks,
                    connections=mp_pose.POSE_CONNECTIONS)
        for landmark in results.pose_landmarks.landmark:
        # 將關鍵點加進list內.
            landmarks.append((int(landmark.x * width), int(landmark.y * height),
                    (landmark.z * width)))    
    
    try:
        # 取得左臀、左膝蓋和左腳踝之間的角度。# 23 25 27
        l_knee2ankle = calculatexy( landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                    landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])
    except:
        l_knee2ankle=0        
    try:
        # 取得右臀、右膝蓋和右腳踝之間的角度。# 24 26 28 
        r_knee2ankle = calculatexy( landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                    landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])
    except:
        r_knee2ankle=0
    return landmarks,image,l_knee2ankle,r_knee2ankle

In [13]:
lastTime1 = 0
cap = cv2.VideoCapture("yoga-1.mp4")
with mp_pose.Pose(static_image_mode=False, 
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5, 
                    model_complexity=0)as pose:

    while (cap.isOpened()):
        # Read feed
        ret, frame = cap.read()
        landmarks,output_image,l_knee2ankle,r_knee2ankle=mediapipe_detection(frame, pose)
        if (time()-lastTime1)>4:
            print([l_knee2ankle,r_knee2ankle])
            lastTime1=time()
        
        cv2.imshow('OpenCV Feed', output_image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            
        # cap.release()
        # cv2.destroyAllWindows()

[96, 93]
[98, 95]
[94, 86]
[99, 101]
[94, 98]
[96, 105]
[8, 12]
[7, 0]
